*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tensorflow
except Exception:
  pass
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense, Dropout, Bidirectional, LSTM
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import re
import nltk
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

# Data Preparation

In [ ]:
df_train=pd.read_csv('train-data.tsv',names=['label','text'], header=0, sep='\t')

In [ ]:
df_train.head()

In [ ]:
df_train.shape

In [ ]:
train_labels = df_train['label'].map({"ham": 0, "spam": 1}).to_numpy()

In [ ]:
train_labels[:10]

In [ ]:
train_texts = df_train['text'].tolist()

In [ ]:
train_texts[:10]

In [ ]:
df_test=pd.read_csv('valid-data.tsv',names=['label','text'], header=0, sep='\t')

In [ ]:
df_test.shape

In [ ]:
df_test.head()

In [ ]:
test_labels = df_test['label'].map({"ham": 0, "spam": 1}).to_numpy()

In [ ]:
test_texts = df_test['text'].tolist()

In [ ]:
train_texts[:10]

In [ ]:
# wordcloud of ham messages
df_all = df_train.append(df_test)
ham_msg = df_all[df_all.label =='ham']
ham_msg_text = " ".join(ham_msg.text.to_numpy().tolist())
ham_msg_cloud = WordCloud(width =520, height =260, stopwords=STOPWORDS,max_font_size=50, background_color ="black", colormap='Blues').generate(ham_msg_text)
plt.figure(figsize=(16,10))
plt.imshow(ham_msg_cloud, interpolation='bilinear')
plt.axis('off') # turn off axis
plt.show()

In [ ]:
# wordcloud of spam messages
spam_msg = df_all[df_all.label =='spam']
spam_msg_text = " ".join(ham_msg.text.to_numpy().tolist())
spam_msg_cloud = WordCloud(width =520, height =260, stopwords=STOPWORDS,max_font_size=50, background_color ="black", colormap='Blues').generate(ham_msg_text)
plt.figure(figsize=(16,10))
plt.imshow(spam_msg_cloud, interpolation='bilinear')
plt.axis('off') # turn off axis
plt.show()

In [ ]:
bar_train = df_train['label'].value_counts()
bar_train

In [ ]:
plt.bar(bar_train.index, bar_train)
plt.xlabel('Label')
plt.title('Number of ham and spam messages in train set')

In [ ]:
bar_test = df_test['label'].value_counts()
bar_test

In [ ]:
plt.bar(bar_test.index, bar_test)
plt.xlabel('Label')
plt.title('Number of ham and spam messages in test set')

#Text preprocessing

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')

In [ ]:
stopwords_eng = set(stopwords.words('english'))
len(stopwords_eng)

In [ ]:
lemmatizer = WordNetLemmatizer()

def clean_txt(txt):
    txt = re.sub(r'([^\s\w])+', ' ', txt)
    txt = " ".join([lemmatizer.lemmatize(word) for word in txt.split()
                    if not word in stopwords_eng])
    txt = txt.lower()
    return txt

In [ ]:
X_train = df_train['text'].apply(lambda x: clean_txt(x))
X_train[:5]

# Tokenization

In [ ]:
# Defining pre-processing hyperparameters
max_len = 50 
trunc_type = "post" 
padding_type = "post" 
oov_tok = "<OOV>" 
vocab_size = 500

In [ ]:
tokenizer = Tokenizer(num_words = vocab_size, char_level=False, oov_token = oov_tok)
tokenizer.fit_on_texts(train_texts)

In [ ]:
# Get the word_index 
word_index = tokenizer.word_index
list(tokenizer.index_word.items())[:20]

In [ ]:
# check how many words 
tot_words = len(word_index)
print(f'There are {tot_words} unique tokens in training data. ')

In [ ]:
# Sequencing and padding on training and testing 
training_sequences = tokenizer.texts_to_sequences(train_texts)
training_padded = pad_sequences(training_sequences, maxlen = max_len, padding = padding_type, truncating = trunc_type )
testing_sequences = tokenizer.texts_to_sequences(test_texts)
testing_padded = pad_sequences(testing_sequences, maxlen = max_len,
padding = padding_type, truncating = trunc_type)

In [ ]:
# Shape of train tensor
print('Shape of training tensor: ', training_padded.shape)
print('Shape of testing tensor: ', testing_padded.shape)

In [ ]:
# Before padding
len(training_sequences[0]), len(training_sequences[1])

In [ ]:
# After padding
len(training_padded[0]), len(training_padded[1])

In [ ]:
print(training_padded[0])

#Spam Detection Model Building

In [ ]:
#LSTM hyperparameters
n_lstm = 20
drop_lstm =0.2

In [ ]:
# Define hyper-parameters
vocab_size = 500 # As defined earlier
embeding_dim = 16
drop_value = 0.2 # dropout
n_dense = 24

In [ ]:
#Model architecture
model = Sequential()
model.add(Embedding(vocab_size, embeding_dim, input_length=max_len))
model.add(Bidirectional(LSTM(n_lstm, dropout=drop_lstm, return_sequences=True)))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
model.compile(loss='binary_crossentropy',optimizer='adam' ,metrics=['accuracy'])

In [ ]:
# fitting a dense spam detector model
num_epochs = 30
early_stop = EarlyStopping(monitor='val_loss', patience=3)
history = model.fit(training_padded, train_labels, epochs=num_epochs, validation_data=(testing_padded, test_labels),callbacks =[early_stop], verbose=2)

In [ ]:
# Model performance on test data 
model.evaluate(testing_padded, test_labels)

In [ ]:
def plot_hist(history):
    import plotly.graph_objects as go
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['accuracy'], name='accuracy', mode='markers+lines'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_accuracy'], name='val_accuracy', mode='markers+lines'))
    fig.update_layout(width=1000, height=500, title='accuracy vs. val accuracy', xaxis_title='Epochs', yaxis_title='accuracy', yaxis_type='log')
    fig.show()

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['loss'], name='loss', mode='markers+lines'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_loss'], name='val_loss', mode='markers+lines'))
    fig.update_layout(width=1000, height=500, title='loss vs. val loss', xaxis_title='Epochs', yaxis_title='loss', yaxis_type='log')
    fig.show()

plot_hist(history)

In [ ]:
def preprocessing(X):
  x = X.apply(lambda x: clean_txt(x))
  x = tokenizer.texts_to_sequences(x)
  return pad_sequences(x, maxlen=max_len)

In [ ]:
print(f"Bi-LSTM architecture loss and accuracy: {model.evaluate(testing_padded, test_labels)} " )

# Evaluating model

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  pred = model.predict(preprocessing(pd.Series([pred_text])))[0]

  return (pred[0], ("ham" if pred[0]<0.5 else "spam"))

pred_text = "wow, is your arm alright. that happened to me one time too"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
